In [1]:
pip install pyresparser

  Obtaining dependency information for srsly>=0.0.7 from https://files.pythonhosted.org/packages/0a/ed/d2c37221fe1975f4b6e8e3cf200d25b905b77e18f6a660b3dc149ade6192/srsly-2.4.8-cp310-cp310-win_amd64.whl.metadata
  Using cached srsly-2.4.8-cp310-cp310-win_amd64.whl.metadata (20 kB)
Using cached srsly-2.4.8-cp310-cp310-win_amd64.whl (481 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.7
    Uninstalling srsly-1.0.7:
      Successfully uninstalled srsly-1.0.7
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from pyresparser import ResumeParser

In [2]:
data = ResumeParser('Curriculum Vitae -Shehan Krishan.pdf').get_extracted_data()

c:\Users\sheha\anaconda3\envs\ADHD\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.9). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
data['skills']

In [ ]:
job_data={
    'Skills_Required': ['python', "machine" "learning", "data analysis"]
}
cv_db = {
    'shehan': 'Curriculum Vitae -Shehan Krishan.pdf', 
}

In [11]:
def calculate_similarity(job_seeker_skills, required_skills):
    seeker_skills = set(job_seeker_skills)
    required_skills = set(required_skills)
    common_skills = seeker_skills.intersection(required_skills)
    similarity_score = round(len(common_skills) / len(required_skills), 1)
    return similarity_score

job_data = {
    'Skills_Required': ['python', 'machine learning', 'data analysis']
}

job_seeker_1_skills = ["python", "data analysis", "database management"]

similarity_score = calculate_similarity(job_seeker_1_skills, job_data['Skills_Required'])
print("Similarity Score:", similarity_score)

Similarity Score: 0.7


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Sample job vacancies data
job_vacancies = pd.DataFrame({
    'Job_Title': ['Data Scientist', 'Software Engineer', 'Web Developer', 'Machine Learning Engineer'],
    'Skills_Required': ['python, machine learning, data analysis', 'java, c++, software development', 'html, css, javascript, web development', 'machine learning, python, data engineering'],
    'Experience_Required': ['3 years', '2 years', '2 years', '4 years']
})

# Sample job seeker's skills and experience
job_seeker_skills = 'html, css, javascript'
job_seeker_experience = '2 years'

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Combine skills and experience into one feature
job_vacancies['Features'] = job_vacancies['Skills_Required'] + ' ' + job_vacancies['Experience_Required']

# Fit and transform the TF-IDF vectorizer with the combined features
tfidf_matrix = tfidf_vectorizer.fit_transform(job_vacancies['Features'])

# Transform job seeker's skills and experience using the TF-IDF vectorizer
job_seeker_profile = tfidf_vectorizer.transform([job_seeker_skills + ' ' + job_seeker_experience])

# Calculate similarity scores
cosine_similarities = linear_kernel(job_seeker_profile, tfidf_matrix).flatten()

# Get job recommendations
job_vacancies['Similarity'] = cosine_similarities
recommended_jobs = job_vacancies.sort_values(by='Similarity', ascending=False).head(3)

print("Recommended Jobs:")
print(recommended_jobs[['Job_Title', 'Skills_Required', 'Experience_Required']])


Recommended Jobs:
           Job_Title                          Skills_Required  \
2      Web Developer   html, css, javascript, web development   
1  Software Engineer          java, c++, software development   
0     Data Scientist  python, machine learning, data analysis   

  Experience_Required  
2             2 years  
1             2 years  
0             3 years  


In [7]:
job_vacancies

Job_Title                             Skills_Required  \
0             Data Scientist     python, machine learning, data analysis   
1          Software Engineer             java, c++, software development   
2              Web Developer      html, css, javascript, web development   
3  Machine Learning Engineer  machine learning, python, data engineering   

  Experience_Required                                           Features  \
0             3 years    python, machine learning, data analysis 3 years   
1             2 years            java, c++, software development 2 years   
2             2 years     html, css, javascript, web development 2 years   
3             4 years  machine learning, python, data engineering 4 y...   

   Similarity  
0    1.000000  
1    0.082569  
2    0.063494  
3    0.733950

In [ ]:
# for get best candinates

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

class getBestCandidate:
    def __init__(self):
        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = None
        self.candidate_skills = None

    def build_model(self, candidate_skills):
        self.candidate_skills = candidate_skills
        self._create_tfidf_matrix()

    def _create_tfidf_matrix(self):
        self.candidate_skills['Features'] = self.candidate_skills['Skills']
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.candidate_skills['Features'])

    def recommend_candidates(self, job_skills, num_recommendations=5):
        job_profile = self.tfidf_vectorizer.transform([job_skills])
        cosine_similarities = linear_kernel(job_profile, self.tfidf_matrix).flatten()
        self.candidate_skills['score'] = cosine_similarities
        recommended_candidates_profile = self.candidate_skills.sort_values(by='score', ascending=False).head(num_recommendations)
        return recommended_candidates_profile[['name', 'Skills', 'score']]

    def save_model(self, file_path):
        with open(file_path, 'wb') as file:
            pickle.dump(self, file)

    @classmethod
    def load_model(cls, file_path):
        with open(file_path, 'rb') as file:
            return pickle.load(file)




In [2]:
Candidates_data = pd.DataFrame({
    'name': ['shehan', 'krishan', 'gayan', 'amal'],
    'Skills': ['python, machine learning, data analysis', 'java, c++, software development', 'html, css, javascript, web development', 'machine learning, python, data engineering'],
})

model = getBestCandidate()
model.build_model(Candidates_data)

In [35]:
model.save_model("score.pkl")

In [36]:
loaded_model = getBestCandidate.load_model("score.pkl")

In [5]:
# Sample job seeker's skills and experience
# job_skills = ["java", "software" "development", "database management"]
job_skills = "java ,software development, database management"
# Get recommendations
recommendations = model.recommend_candidates(job_skills)
print("Recommended Jobs:")
print(recommendations)

Recommended Jobs:
      name                                      Skills     score
1  krishan             java, c++, software development  1.000000
2    gayan      html, css, javascript, web development  0.178578
0   shehan     python, machine learning, data analysis  0.000000
3     amal  machine learning, python, data engineering  0.000000


# collaborative

In [19]:
import pandas as pd
import random

# Job Titles and their respective skills
job_titles = {
    "Software Engineer": ["java", "python", "c++", "software development", "algorithms"],
    "Web Developer": ["html", "css", "javascript", "web development", "frontend", "backend"],
    "Data Scientist": ["python", "r", "machine learning", "data analysis", "statistics"],
    "Database Administrator": ["sql", "database management", "data modeling", "oracle", "mysql"],
    "Network Engineer": ["networking", "ccna", "cisco", "routing", "switching"]
}

# Generating user data
user_data = {
    "ID": [],
    "Skills": [],
    "job_vacancies_Job_ID": [],
    "Job_Title": []
}

for i in range(50):
    user_id = random.randint(1, 20)
    job_id = random.choice([0, 1, 2, 3, 4])
    job_title = list(job_titles.keys())[job_id]
    skills = random.sample(job_titles[job_title], random.randint(1, len(job_titles[job_title])))
    user_data["ID"].append(user_id)
    user_data["Skills"].append(skills)
    user_data["job_vacancies_Job_ID"].append(job_id)
    user_data["Job_Title"].append(job_title)

user_df = pd.DataFrame(user_data)
print(user_df)


    ID                                             Skills  \
0    9               [networking, cisco, ccna, switching]   
1    7  [data modeling, oracle, sql, database management]   
2    3  [python, r, statistics, machine learning, data...   
3    5    [software development, c++, algorithms, python]   
4    7                                           [python]   
5   12             [python, machine learning, statistics]   
6    8                [python, statistics, data analysis]   
7   10                                             [java]   
8   16  [css, web development, html, javascript, backe...   
9   20                                       [javascript]   
10   1                               [algorithms, python]   
11  18               [python, software development, java]   
12  10                 [oracle, sql, database management]   
13  18                                             [java]   
14  14  [css, html, backend, javascript, web developme...   
15  19      [frontend, h

In [20]:
unique_ids = user_df['ID'].unique()
print(unique_ids)

[ 9  7  3  5 12  8 10 16 20  1 18 14 19  4 11  6 17 13 15]


In [21]:
id_counts = user_df['ID'].value_counts()
print(id_counts)

ID
11    5
4     4
12    4
8     4
9     3
16    3
18    3
15    3
10    3
5     3
20    2
7     2
19    2
3     2
17    2
13    2
14    1
6     1
1     1
Name: count, dtype: int64


In [22]:
user_df

ID                                             Skills  \
0    9               [networking, cisco, ccna, switching]   
1    7  [data modeling, oracle, sql, database management]   
2    3  [python, r, statistics, machine learning, data...   
3    5    [software development, c++, algorithms, python]   
4    7                                           [python]   
5   12             [python, machine learning, statistics]   
6    8                [python, statistics, data analysis]   
7   10                                             [java]   
8   16  [css, web development, html, javascript, backe...   
9   20                                       [javascript]   
10   1                               [algorithms, python]   
11  18               [python, software development, java]   
12  10                 [oracle, sql, database management]   
13  18                                             [java]   
14  14  [css, html, backend, javascript, web developme...   
15  19      [frontend, html, javascript, web development]   
16   4                                    [data modeling]   
17  11                                             [java]   
18   8               [java, software development, python]   
19   5  [java, software development, algorithms, pytho...   
20   6                        [networking, routing, ccna]   
21  11                  [machine learning, statistics, r]   
22  11  [css, html, web development, javascript, backe...   
23   3    [python, c++, software development, algorithms]   
24  12                 [ccna, networking, cisco, routing]   
25  18               [ccna, cisco, networking, switching]   
26  17                               [data modeling, sql]   
27  16                              [database management]   
28   8                         [sql, database management]   
29  19      [ccna, networking, switching, routing, cisco]   
30  20  [database management, mysql, sql, data modelin...   
31  10                                    [data modeling]   
32   4  [algorithms, software development, c++, python...   
33   9  [javascript, html, css, backend, web development]   
34   5            [algorithms, software development, c++]   
35  13                                           [python]   
36   4                       [java, software development]   
37  11                                             [ccna]   
38  12      [ccna, switching, cisco, networking, routing]   
39  15                             [cisco, routing, ccna]   
40  16    [software development, python, algorithms, c++]   
41  15                                        [switching]   
42   8                              [networking, routing]   
43   4  [database management, sql, data modeling, oracle]   
44  15                                           [oracle]   
45  13  [web development, backend, javascript, css, fr...   
46  17  [database management, sql, data modeling, mysq...   
47  11                                              [c++]   
48   9                                           [python]   
49  12                     [switching, cisco, networking]   

    job_vacancies_Job_ID               Job_Title  
0                      4        Network Engineer  
1                      3  Database Administrator  
2                      2          Data Scientist  
3                      0       Software Engineer  
4                      0       Software Engineer  
5                      2          Data Scientist  
6                      2          Data Scientist  
7                      0       Software Engineer  
8                      1           Web Developer  
9                      1           Web Developer  
10                     0       Software Engineer  
11                     0       Software Engineer  
12                     3  Database Administrator  
13                     0       Software Engineer  
14                     1           Web Developer  
15                     1           Web Developer  
16                     3 

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Convert list of skills to text
user_df['Skills_text'] = user_df['Skills'].apply(lambda x: ' '.join(x))

# Convert Job_Title to target
y = user_df['Job_Title']

# Convert Skills_text to features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(user_df['Skills_text'])

# Train Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X, y)

def recommend_job_title(skills):
    # Convert skills to text
    skills_text = ' '.join(skills)
    
    # Transform skills to features
    skills_features = vectorizer.transform([skills_text])
    
    # Predict job title
    predicted_title = clf.predict(skills_features)
    
    return predicted_title[0]


In [44]:
# Example usage
skills = ["machine learning"]
recommended_title = recommend_job_title(skills)
print("Recommended Job Title based on skills {}: {}".format(skills, recommended_title))

Recommended Job Title based on skills ['machine learning']: Data Scientist


In [48]:
user_df.to_csv("dataUser.csv")

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert list of skills to text
# user_df['Skills_text'] = user_df['Skills'].apply(lambda x: ' '.join(x))

# Convert Skills_text to features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(user_df['Skills_text'])

def recommend_job_title_cosine_similarity(skills):
    # Convert skills to text
    skills_text = ' '.join(skills)

    # Transform skills to features
    skills_features = vectorizer.transform([skills_text])

    # Calculate cosine similarity between input skills and job skills
    cosine_similarities = cosine_similarity(skills_features, X)

    # Get indices of top similar job titles
    similar_jobs_indices = cosine_similarities.argsort()[0][::-1]

    # Get top 5 similar job titles
    top_similar_jobs = user_df.iloc[similar_jobs_indices[:5]]['Job_Title'].unique()

    return top_similar_jobs

# Example usage
skills = ["python", "data analysis", "machine learning"]
recommended_titles = recommend_job_title_cosine_similarity(skills)
print("Recommended Job Titles based on skills {} using Cosine Similarity: {}".format(skills, recommended_titles))


Recommended Job Titles based on skills ['python', 'data analysis', 'machine learning'] using Cosine Similarity: ['Data Scientist']


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert list of skills to text
user_df['Skills_text'] = user_df['Skills'].apply(lambda x: ' '.join(x))

# Convert Skills_text to features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(user_df['Skills_text'])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, user_df['Job_Title'], test_size=0.2, random_state=42)

# Train K-Nearest Neighbors classifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

def recommend_job_title_knn(skills):
    # Transform skills to features
    skills_features = vectorizer.transform([' '.join(skills)])

    # Predict job title
    predicted_title = knn.predict(skills_features)

    return predicted_title[0]




NameError: name 'user_df' is not defined

In [64]:
def recommend_job_title_knn_acc(skills):
    # Transform skills to features
    # skills_features = vectorizer.transform([' '.join(skills)])

    # Predict job title
    predicted_title = knn.predict(skills)

    return predicted_title[0]

In [65]:
y_pred = [recommend_job_title_knn_acc(skills) for skills in X_test]

In [66]:
accuracy = accuracy_score(y_test, y_pred)
print("Model: K-Nearest Neighbors, Accuracy: {:.2f}".format(accuracy))

Model: K-Nearest Neighbors, Accuracy: 0.90


In [63]:
# Example usage
skills = ["python", "data analysis", "machine learning"]
recommended_title = recommend_job_title_knn(skills)
print("Recommended Job Title based on skills {} using K-Nearest Neighbors: {}".format(skills, recommended_title))

Recommended Job Title based on skills ['python', 'data analysis', 'machine learning'] using K-Nearest Neighbors: Data Scientist


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

user_df = pd.read_csv("dataUser.csv")

# Convert Skills_text to features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(user_df['Skills_text'])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, user_df['Job_Title'], test_size=0.2, random_state=42)

# Train Random Forest classifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

def recommend_job_title_rf(skills):
    # Transform skills to features
    skills_features = vectorizer.transform([' '.join(skills)])

    # Predict job title
    predicted_title = rf.predict(skills_features)

    return predicted_title[0]

def recommend_job_title_rf_acc(skills):
    # Predict job title
    predicted_title = rf.predict(skills)

    return predicted_title[0]

# Test the model
y_pred = [recommend_job_title_rf_acc(skills) for skills in X_test]
accuracy = accuracy_score(y_test, y_pred)
print("Model: Random Forest, Accuracy: {:.2f}".format(accuracy))

Model: Random Forest, Accuracy: 0.90


In [6]:
import joblib

# Save the model and vectorizer
joblib.dump(rf, 'random_forest_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [11]:
# Load the model and vectorizer
rfx = joblib.load('random_forest_model.pkl')
vectorizerx = joblib.load('tfidf_vectorizer.pkl')


In [12]:
def recommend_job_title_rf_acc(skills):
    # Transform skills to features
    skills_features = vectorizerx.transform([' '.join(skills)])

    # Predict job title
    predicted_title = rfx.predict(skills_features)

    return predicted_title[0]


In [13]:
# Example usage
skills = ["python", "data analysis", "machine learning"]
recommended_title = recommend_job_title_rf_acc(skills)
print("Recommended Job Title based on skills {} using Random Forest: {}".format(skills, recommended_title))


Recommended Job Title based on skills ['python', 'data analysis', 'machine learning'] using Random Forest: Data Scientist


In [5]:
# Example usage
skills = ["python", "data analysis", "machine learning"]
recommended_title = recommend_job_title_rf(skills)
print("Recommended Job Title based on skills {} using Random Forest: {}".format(skills, recommended_title))

Recommended Job Title based on skills ['python', 'data analysis', 'machine learning'] using Random Forest: Data Scientist
